<a href="https://colab.research.google.com/github/OnePassio/CE888/blob/master/lab_5_my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 5
https://github.com/albanda/CE888/tree/master/lab5-recommender

1.Load Database "jester-data-1.csv"
Format:
- Ratings are real values ranging from -10.00 to +10.00 (the value "99" corresponds to "null" = "not rated").
- One row per user
- The first column gives the number of jokes rated by that user. The next 100 columns give the ratings for jokes 01 - 100.
- The sub-matrix including only columns {5, 7, 8, 13, 15, 16, 17, 18, 19, 20} is dense. Almost all users have rated those jokes (see discussion of "universal queries" in the above paper).

In [0]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


df_data= pd.read_excel('https://raw.githubusercontent.com/OnePassio/CE888/master/lab5/jester-data-1.xls',
                       header=None)
df_data.head()

2.Using the helper function provided in the helper_function notebook (or create your own), label 10% of the dataset cells as 99. This is your validation set. Keep the the actual values of the cells so you can use them later.

In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(orig!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
# original data
data_original=df_data.values
print(data_original.shape)

#rated = np.where(data_original!=99)
#print(len(rated[0]), rated[1].shape)

In [0]:
#new data with 10% replace value 99
data_new, idx = replace(data_original, 0.1)
print(data_new.shape)
data_new

3.Use latent factor modeling to infer the hidden ratings of the users (they are labeled as "99" in the dataset) on the training set

In [0]:
#data without first column
data_new_2 = np.delete(data_new, 0, 1)
#print(data_new_2.shape)
#data_new

# Initialise as random values
n_latent_factors = 2
latent_user_preferences = np.random.random((data_new_2.shape[0], n_latent_factors))
latent_item_features = np.random.random((data_new_2.shape[1], n_latent_factors))



In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=30,print_value=10):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = data_new_2[user_id][item_id]
                if abs(rating-99)>0.1: # rating is not 99.0  -->not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        #if (iteration % print_value) == 0:
          #print("mse: {}".format(mse))
        print("iteration: {},mse: {}".format(iteration,mse))

In [0]:
sgd(30,10)